In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
"""
Load data and setup train/test split
~91k train reviews, ~39k test
"""
reviews = pd.read_csv("Data/classifier_data.csv")
reviews.dropna(inplace=True)
train, test = train_test_split(reviews, test_size=0.30, random_state=12)
train_x = train['Review']
train_y = train['Real']
test_x = test['Review']
test_y = test['Real']

In [3]:
"""
Dummy Classifier to be used as a baseline for comparison
The accuracy is 50% which is expected as the data is 50% fake reviews
"""
from sklearn.dummy import DummyClassifier
baseline = DummyClassifier(strategy="stratified")
baseline.fit(train_x, train_y)

np.mean(baseline.predict(test_x) == test_y)*100

50.064135454079015

In [57]:
"""
Vectorize the reviews in order to use a classifier
tfidf helps reduce impact of common words
"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pipe = Pipeline([('count_vect',CountVectorizer()),
                ('tfidf_transformer',TfidfTransformer()),
                ('nbClassifier',MultinomialNB())])
pipe.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('count_vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf_transformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('nbClassifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [71]:
preds = pipe.predict(test_x)

In [72]:
print(classification_report(test_y, preds))


              precision    recall  f1-score   support

           0       0.64      0.51      0.57     19420
           1       0.60      0.71      0.65     19560

    accuracy                           0.61     38980
   macro avg       0.62      0.61      0.61     38980
weighted avg       0.62      0.61      0.61     38980



In [4]:
"""
Load LSTM model and predict on unseen test data
Model was trained in file "text_classifier_lstm.ipynb"
"""
model = tf.keras.models.load_model('models/lstm')

In [47]:
test_data = test.Review.values

In [40]:
#predictions = model.predict(np.array(test_data))
#preds = np.where(predictions > 0.5, 1, 0)

In [48]:
test_loss, test_acc = model.evaluate(test_data, test.Real.values)

1219/1219 [==============================] - 17s 11ms/step - loss: 0.6612 - accuracy: 0.6057


In [49]:
test_loss

0.6611835956573486

In [50]:
test_acc

0.6056695580482483